In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from datetime import timedelta
from sklearn.linear_model import LogisticRegression

In [2]:
brand = pd.read_csv('../data_concat/movies.csv') 

fb_1k = pd.read_table('../data/view_brand_rollup_facebook_1k-10k.tsv')
fb_10k = pd.read_table('../data/view_brand_rollup_facebook_10kplus.tsv')
fb = pd.concat([fb_1k, fb_10k])
fb = pd.merge(fb, brand, left_on = 'movie_id', right_on = 'brand_ods_id', how = 'left')

insta_1k = pd.read_table('../data/view_brand_rollup_instagram_1k-10k.tsv')
insta_10k = pd.read_table('../data/view_brand_rollup_instagram_10kplus.tsv')
insta = pd.concat([insta_1k, insta_10k])
insta = pd.merge(insta, brand, left_on = 'movie_id', right_on = 'brand_ods_id', how = 'left')

twit_1k = pd.read_table('../data/view_brand_rollup_twitter_1k-10k.tsv')
twit_10k = pd.read_table('../data/view_brand_rollup_twitter_10kplus.tsv')
twit = pd.concat([twit_1k, twit_10k])
twit = pd.merge(twit, brand, left_on = 'movie_id', right_on = 'brand_ods_id', how = 'left')

In [3]:
fb['data_for']= pd.to_datetime(fb['data_for']) 
fb['released_on'] = pd.to_datetime(fb['released_on'])
fb['days_after_release'] = fb['data_for'] - fb['released_on'] 
fb = fb[(fb['days_after_release'] <= '0 days') & (fb['days_after_release'] >= '-365 days')]
fb['days'] = fb['days_after_release'] + timedelta(days=365)

insta['data_for']= pd.to_datetime(insta['data_for']) 
insta['released_on'] = pd.to_datetime(insta['released_on'])
insta['days_after_release'] = insta['data_for'] - insta['released_on'] 
insta = insta[(insta['days_after_release'] <= '0 days') & (insta['days_after_release'] >= '-365 days')]
insta['days'] = insta['days_after_release'] + timedelta(days=365)

twit['data_for']= pd.to_datetime(twit['data_for']) 
twit['released_on'] = pd.to_datetime(twit['released_on'])
twit['days_after_release'] = twit['data_for'] - twit['released_on'] 
twit = twit[(twit['days_after_release'] <= '0 days') & (twit['days_after_release'] >= '-365 days')]
twit['days'] = twit['days_after_release'] + timedelta(days=365)

In [4]:
# comments = pd.read_csv('../data_concat/comments_with_movies.csv')

In [5]:
movies = pd.read_csv('movie_data_w_bo.csv').dropna()
movies['ProductionBudget'] = movies['ProductionBudget'].str.replace('[b\$,\']', '').astype('int64')
movies['WorldwideGross'] = movies['WorldwideGross'].str.replace('[b\$,\']', '').astype('int64')

movie = pd.read_csv('../data_concat/movies.csv')

dat = pd.merge(movie, movies, left_on = 'replace', right_on = 'Movie', how = 'left')


dat['profit'] = dat['WorldwideGross'] - dat['ProductionBudget']

def success(profit):
    if profit < 0:
        return False
    else:
        return True
    
dat['success'] = dat['profit'].map(success)

df = dat.drop(['X1', 'data_profile_dcs_uid', 'data_profile_channel_type', 'data_profile_source_type',\
              'data_affiliation_active_flag', 'data_affiliation_start_date_key', 'data_affiliation_end_date_key',\
              'budget', 'ticker_symbol', 'Unnamed: 0', 'date URL', \
              'ReleaseDate', 'summary URL', 'Movie'], axis = 1)
df = df.dropna()

### Add total_post_interaction for prediction

In [6]:
month_filter = list(np.arange(30, 365, 30))
days_filter = ['-' + str(s) + ' days' for s in month_filter]

fb_month = fb[fb['days_after_release'].isin(days_filter)]
insta_month = insta[insta['days_after_release'].isin(days_filter)]
twit_month = twit[twit['days_after_release'].isin(days_filter)]

fb_month = fb_month.pivot(index = 'movie_id', columns = 'days_after_release',  values=['total_post_interactions'])
insta_month = insta_month.pivot(index = 'movie_id', columns = 'days_after_release',  values=['total_post_interactions'])
twit_month = twit_month.pivot(index = 'movie_id', columns = 'days_after_release',  values=['total_post_interactions'])

fb_month.columns = fb_month.columns.droplevel(0)
fb_month.columns = [s.replace(' ', '') + '_fb' for s in days_filter]
insta_month.columns = insta_month.columns.droplevel(0)
insta_month.columns = [s.replace(' ', '') + '_insta' for s in days_filter]
twit_month.columns = twit_month.columns.droplevel(0)
twit_month.columns = [s.replace(' ', '') + '_twit' for s in days_filter]

df = pd.merge(df, fb_month, left_on = 'brand_ods_id', right_on = 'movie_id')
df = pd.merge(df, insta_month, left_on = 'brand_ods_id', right_on = 'movie_id')
df = pd.merge(df, twit_month, left_on = 'brand_ods_id', right_on = 'movie_id')

In [ ]:
df

### Add Sentiment